In [60]:
import torch
import numpy as np
from math import sin, cos

SENTENCE = "<s> I LIKE PIE VERY MUCH PIE ."
DIM_EMB = 512
torch.manual_seed(7)

vocab = dict()
i = 0
for token in SENTENCE.split():
    if token not in vocab:
        vocab[token] = i
        i += 1
vocab["<unk>"] = i
vocab["Peter"] = i + 1
print(vocab)

inv_vocab = dict()
for word in vocab:
    inv_vocab[vocab[word]] = word

DICT_SIZE = len(vocab)

oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

for i in range(len(SENTENCE.split())):
    word = SENTENCE.split()[i]
    oneHotSent[i][vocab[word]] = 1

oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
class FUCKYOU(torch.nn.Module):
    def __init__(self, vocab_size, batch_size):
        super().__init__()
        self.DICT_SIZE = vocab_size
        self.batch_size = batch_size #@TODO CHANGE
        self.transformerHeadConcat = 64 #@TODO CHANGE
        
        Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wq = torch.nn.Parameter(Wq_skeleton, requires_grad=True)

        Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wk = torch.nn.Parameter(Wk_skeleton, requires_grad=True)

        Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wv = torch.nn.Parameter(Wv_skeleton, requires_grad=True)
        
        Wo_skeleton = torch.tensor(np.zeros((self.transformerHeadConcat, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wo = torch.nn.Parameter(Wo_skeleton, requires_grad=True)
        
        We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.We = torch.nn.Parameter(We_skeleton, requires_grad=True)
        
        Wff_skeleton = torch.tensor(np.zeros((DIM_EMB, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wff = torch.nn.Parameter(Wff_skeleton, requires_grad=True)
        Bff_skeleton = torch.tensor(np.zeros((DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Bff = torch.nn.Parameter(Bff_skeleton, requires_grad=True)
        
        Wproj_skeleton = torch.tensor(np.zeros((DIM_EMB, DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wproj = torch.nn.Parameter(Wproj_skeleton, requires_grad=True)
        Bproj_skeleton = torch.tensor(np.zeros((DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Bproj = torch.nn.Parameter(Bproj_skeleton, requires_grad=True)
        
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        
    
    #DATA PREPARATION DONE

    # Create self attention
    # also need to add a mask here
    # @TODO Add a Bias to all Wsz
    def add_transformer_head_with_mask(self, DIM_EMB, Batch, mask):
        Q = Batch.mm(self.Wq)
        V = Batch.mm(self.Wv)
        K = Batch.mm(self.Wk)

        tmp1 = Q.mm(torch.t(K))

        tmp = tmp1/(64**(1/2))
        #print(tmp)

        softM = torch.nn.Softmax(dim=-1)
        #apply maks here
        tmp = tmp.masked_fill(mask == 0, -1e9)

        a = softM(tmp)

        z = a.mm(V)
        return z

    # @TODO add a bias to Wo
    def addLinearProj(self, concatTransHead, DIM_EMB):
        '''Already concatenataed all heads'''

        return concatTransHead.mm(self.Wo)

    #Normalize
    def layerNorm(self, mat):
        return (mat - mat.mean(dim=0, keepdim=True)) / (mat.std(dim=0, keepdim=True))

    EPOCHS=10
    def fwd(self):
        
        #Positional embeddings here
        Batch = oneHotTorch.mm(self.We)
        Pos_emb = np.zeros(Batch.shape, dtype=float)
        for p in range(len(SENTENCE.split())):
            for i in range(DIM_EMB//2):
                val = p/(10000**(2*i/DIM_EMB))
                Pos_emb[p][2*i] = sin(val)
                Pos_emb[p][2*i + 1] = cos(val)

        Pos_emb_var = torch.nn.Parameter(torch.tensor(Pos_emb, dtype=torch.float32), requires_grad=False)

        Batch = Batch*(DIM_EMB**(1/2))

        Batch = Batch + Pos_emb_var

        #Batch (in this case a one sentence batch) is prepared. Maxi calls this embedded sentence

        #Create MASK HERE
        #
        mask = np.triu(np.ones((len(SENTENCE.split()), len(SENTENCE.split()))), k=1)
        mask = torch.from_numpy(mask) == 0

        transformer_head = self.add_transformer_head_with_mask(DIM_EMB, Batch, mask)
        postAttention = self.addLinearProj(transformer_head, DIM_EMB)

        # Add residual connection to the embeddings
        postAttention = postAttention + Batch

        normalizedPostAttention = self.layerNorm(postAttention)

        # @TODO no dropout
        # Take the output of the attention and do a one layer FF that outputs size of
        activation = torch.nn.ReLU()
        postFF = activation(normalizedPostAttention.mm(self.Wff) + self.Bff)

        #Residual
        preSoftmaxProj = self.layerNorm(postFF + normalizedPostAttention)

        #Projexct

        linearLay = activation(preSoftmaxProj.mm(self.Wproj) + self.Bproj)

        #Softmax
        softmax = torch.nn.LogSoftmax(dim=1)
        output = softmax(linearLay)
        return output

    #@TODO
    def getOneHot(self, truth):
        ret = torch.Tensor(self.batch_size).long() 

model = FUCKYOU(DICT_SIZE, len(SENTENCE.split()))
EPOCHS=10

def printSent(softmaxd, vocab):
    words = []
    for i in range(len(softmaxd)):
        _, idx = torch.max(fwd[i], 0)
        words.append(vocab[int(idx)])
    print(" ".join(words))

for e in range(EPOCHS):
    model.optimizer.zero_grad()
    fwd = model.fwd()
    printSent(fwd, inv_vocab)
    truth = oneHotTorch.long() #SAKA [:, 1:].contiguous().view(-1)
    truth = torch.Tensor([0, 1, 2, 3, 4, 5, 3, 6]).long()
    loss = torch.nn.functional.cross_entropy(fwd, truth)
    print("Loss: ", loss.item())
    loss.backward()
    model.optimizer.step()

    



{'<s>': 0, 'I': 1, 'LIKE': 2, 'PIE': 3, 'VERY': 4, 'MUCH': 5, '.': 6, '<unk>': 7, 'Peter': 8}
<s> VERY LIKE VERY PIE <s> VERY <s>
Loss:  17.468599319458008
<s> I LIKE I PIE <s> VERY <s>
Loss:  9.194619178771973
. PIE LIKE PIE <s> . VERY .
Loss:  5.541458606719971
. PIE LIKE PIE <s> . PIE .
Loss:  3.3299055099487305
. I LIKE I Peter . PIE .
Loss:  2.780008316040039
VERY I LIKE I Peter VERY PIE VERY
Loss:  2.9167559146881104
<s> PIE LIKE PIE Peter <s> PIE <s>
Loss:  2.917109966278076
<s> I LIKE PIE Peter <s> PIE <s>
Loss:  2.602118968963623
<s> I LIKE I Peter <s> PIE <s>
Loss:  2.1098036766052246
. I LIKE I Peter . PIE .
Loss:  2.078972816467285
